In [113]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt
from testCases import *
import sklearn
import sklearn.datasets
import sklearn.linear_model
from planar_utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset
%matplotlib inline

np.random.seed(1) # set a seed so that the results are consistent

In [114]:
def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    n_x = X.shape[0]
    n_h = 4
    n_y = Y.shape[0]
    return (n_x, n_h, n_y)

In [115]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)
    b2 = np.zeros((n_y, 1))
#     print(W1)
#     print(b1)
#     print(W2)
#     print(b2
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    return parameters

In [116]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    ### START CODE HERE ### (≈ 4 lines of code)
    Z1 = np.dot(W1, X) #(n_h, n_x) * (n_x, m) = (n_h, m)
    A1 = np.tanh(Z1)  #(n_h, m)
    Z2 = np.dot(W2, A1) #(n_y, n_h) * (n_h, m) = (n_y, m)
    A2 = sigmoid(Z2)
    ### END CODE HERE ###
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [117]:
# GRADED FUNCTION: compute_cost

def compute_cost(A2, Y, parameters):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    """
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    ### START CODE HERE ### (≈ 2 lines of code)
    cost = np.sum(np.dot(Y, np.log(A2).T) + np.dot((1 - Y), np.log(1 - A2).T)) / -m
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost

In [118]:
# GRADED FUNCTION: backward_propagation

def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### (≈ 2 lines of code)
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### (≈ 2 lines of code)
    A1 = cache["A1"]            #(n_h, m)
    A2 = cache["A2"]            #(n_y, m)
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### (≈ 6 lines of code, corresponding to 6 equations on slide above)
    dZ2 = A2 - Y                #(n_y, m)
    dW2 = np.dot(dZ2, A1.T) / m  #(n_y, n_h)
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = 1.0 / m * np.dot(dZ1, X.T)
    db1 = 1.0 / m * np.sum(dZ1, axis=1, keepdims=True)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [119]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate = 0.009):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### (≈ 4 lines of code)
    dW1 = grads["dW1"]
    dW2 = grads["dW2"]
    db1 = grads["db1"]
    db2 = grads["db2"]
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 -= learning_rate * dW1
    W2 -= learning_rate * dW2
    b1 -= learning_rate * db1
    b2 -= learning_rate * db2
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [120]:
def nn_model(X, Y, n_h, num_iterations = 2000, print_cost = True):
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']
    
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(X, parameters)
        
        cost = compute_cost(A2, Y, parameters)
        
        grads = backward_propagation(parameters, cache, X, Y)
        
        parameters = update_parameters(parameters, grads)
        
        if(print_cost is True and i % 1000 is 0):
            print("Cost after iteration %i: %f" %(i, cost))
    return parameters

In [121]:
# GRADED FUNCTION: predict
# after learning from the dataset
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    A2, cache = forward_propagation(X, parameters)
    predictions = np.array([1 if x>0.5 else 0 for x in A2.reshape(-1, 1)])
    ### END CODE HERE ###
    
    return predictions

In [122]:
# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [123]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T 
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.
print train_set_x.shape
print train_set_y.shape


(12288, 209)
(1, 209)


In [125]:
parameters = nn_model(train_set_x, train_set_y, n_h = 3, num_iterations=1000)
result = predict(parameters, train_set_x)

Cost after iteration 0: 0.723383


In [126]:
# Print accuracy
predictions = predict(parameters, train_set_x)
print ('Accuracy: %d' % float((np.dot(train_set_y,predictions.T) + np.dot(1-train_set_y,1-predictions.T))/float(train_set_y.size)*100) + '%')

Accuracy: 98%


In [127]:
def test_my_set(imgs, parameters):
    result = []
    for my_image in imgs:
        # We preprocess the image to fit your algorithm.
        fname = my_image
        image = np.array(ndimage.imread(fname, flatten=False))
        my_image = scipy.misc.imresize(image, size=(64, 64)).reshape((1, 64*64*3)).T
        my_predicted_image = predict(parameters, my_image)
        if my_predicted_image >= 0.5:
            result.append("cat")
        else:
            result.append("nocat")
    return result

In [128]:
import os
import os.path
def get_img_set(root_path):
    imgs = []
    for parent, dirnames, filenames in os.walk(root_path):
        for filename in filenames:
            imgs.append(root_path + filename)
    return imgs

In [129]:
import json
def store(data):
    with open('parameters.json', 'w') as json_file:
        json_file.write(json.dumps(data))

def load():
    with open('parameters.json') as json_file:
        data = json.load(json_file)
        return data
    
# class DateEncoder(json.JSONEncoder ):  
#     def default(self, obj):  
#         if isinstance(obj, np.array):  
#             return obj.__str__()  
#         return json.JSONEncoder.default(self, obj)  

# xxx = [[1], [2], [3], [4]]
# print xxx
# zzz = np.dot(xxx, [[1, 2, 3, 4]])
# zzz = zzz.reshape(1, -1).tolist()
# print zzz
# json_1 = {'num':1112, 'date':zzz}
# print json.dumps(json_1, cls=DateEncoder)

In [131]:
root_path = "/home/sar/Desktop/My_test/Python/dog_set/"
data = {}
data["W1"] = parameters["W1"].reshape(1, -1).tolist()
data["W2"] = parameters["W2"].reshape(1, -1).tolist()
data["b2"] = parameters["b2"].tolist()
data["b1"] = parameters["b1"].tolist()
# store(data)
print parameters["W1"].shape
print parameters["W2"].shape
print parameters["b1"].shape
print parameters["b2"].shape

# data_ = load()
# loaded_parameters = {}
# loaded_parameters["W1"] = np.array(data_["W1"]).reshape(parameters["W1"].shape)
# loaded_parameters["W2"] = np.array(data_["W2"]).reshape(parameters["W2"].shape)
# loaded_parameters["b1"] = np.array(data_["b1"]).reshape(parameters["b1"].shape)
# loaded_parameters["b2"] = np.array(data_["b2"]).reshape(parameters["b2"].shape)
# result = test_my_set(get_img_set(root_path), loaded_parameters)
# print result
# a = 0
# for label in result:
#     if label == 'nocat':
#         a += 1
# print "accuracy: " + str(1.0 * a / len(result))

(3, 12288)
(1, 3)
(3, 1)
(1, 1)
